In [75]:
# Dependencies and Setup
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Import the datetime module from the datetime library.
from datetime import datetime
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import API key
from config import g_key
import ipywidgets as widgets

In [76]:
# Store the CSV you saved created in part one into a DataFrame.
hotel_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
hotel_df.head(20)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Sfantu Gheorghe,RO,55.81,overcast clouds,45.87,25.78,Hotel Park Sfantu Gheorghe
1,5,Lazi,UA,53.38,light rain,48.07,24.23,Olenka
2,6,Coihaique,CL,46.40,light rain,-45.58,-72.07,Diego De Almagro Coyhaique
3,7,Dunedin,NZ,57.99,overcast clouds,-45.87,170.50,Scenic Hotel Southern Cross
4,8,Tuatapere,NZ,53.78,clear sky,-46.13,167.68,Tom Cruse Refreshment Company
5,12,Frankfort,ZA,46.24,clear sky,-27.27,28.48,Formule 11 Accomodation
6,15,Mataura,NZ,52.38,overcast clouds,-46.19,168.86,Ellie's Villa
7,16,Valladolid,ES,52.00,broken clouds,41.58,-4.67,Hotel Ruta del Duero
8,18,Lebu,CL,51.39,broken clouds,-37.62,-73.65,Hostal Las Lilas
9,20,Pacifica,US,59.00,mist,37.61,-122.49,"Pacifica Lighthouse, Trademark Collection by W..."


In [102]:
city1_df=hotel_df.loc[(hotel_df["City"] =="Bluff")]
city2_df=hotel_df.loc[(hotel_df["City"] =="Dunedin")]
city3_df=hotel_df.loc[(hotel_df["City"] =="Kaitangata")]
city4_df=hotel_df.loc[(hotel_df["City"] =="Mataura")]

city_all_df=hotel_df.loc[((hotel_df["City"] == "Bluff") | (hotel_df["City"] == "Dunedin") | (hotel_df["City"] == "Kaitangata") | (hotel_df["City"] == "Mataura"))]

#Create Infobox
direction_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

direction_info = [direction_box_template.format(**row) for index, row in city_all_df.iterrows()]
direction_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Scenic Hotel Southern Cross</dd>\n<dt>City</dt><dd>Dunedin</dd>\n<dt>Country</dt><dd>NZ</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 57.99°F</dd>\n</dl>\n',
 "\n<dl>\n<dt>Hotel Name</dt><dd>Ellie's Villa</dd>\n<dt>City</dt><dd>Mataura</dd>\n<dt>Country</dt><dd>NZ</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 52.38°F</dd>\n</dl>\n",
 '\n<dl>\n<dt>Hotel Name</dt><dd>Kaitangata Motor Camp</dd>\n<dt>City</dt><dd>Kaitangata</dd>\n<dt>Country</dt><dd>NZ</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 55.0°F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Bluff Homestead - Guesthouse & Campervan Park</dd>\n<dt>City</dt><dd>Bluff</dd>\n<dt>Country</dt><dd>NZ</dd>\n<dt>Current Weather</dt><dd>clear sky and 52.36°F</dd>\n</dl>\n']

In [100]:
dir_locations = city_all_df[["Lat", "Lng"]]
dir_locations


,Lat,Lng
3,-45.87,170.50
6,-46.19,168.86
17,-46.28,169.85
19,-46.60,168.33


In [103]:
# Latitude-longitude pairs
city1_loc=(city1_df['Lat'].max(), city1_df['Lng'].max())
city2_loc=(city2_df['Lat'].max(), city2_df['Lng'].max())
city3_loc=(city3_df['Lat'].max(), city3_df['Lng'].max())
city4_loc=(city4_df['Lat'].max(), city4_df['Lng'].max())
dir_locations = city_all_df[["Lat", "Lng"]]

#city 1 Infobox
#Create Infobox
info_box_city1 = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

fig = gmaps.figure()
directions = gmaps.directions_layer(city1_loc, city2_loc, waypoints=[city3_loc, city4_loc], travel_mode='DRIVING')
dir_marker_layer = gmaps.marker_layer(dir_locations, info_box_content=direction_info)
fig.add_layer(dir_marker_layer)
fig.add_layer(directions)
fig

Figure(layout=FigureLayout(height='420px'))

In [104]:
fig = gmaps.figure()
dir_marker_layer = gmaps.marker_layer(dir_locations, info_box_content=direction_info)
fig.add_layer(dir_marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))